In [2]:
%cd ..


C:\Users\benza-ord\source\repos\CS583-Research-Project


In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv("cxs583_cleaned_file_v2.csv")
df.head()

,date,time,tweet,class,result,tweet_length,tweet_cleaned,tweet_clst
0,2012-10-16 00:00:00,10:28:53-05:00,"Kirkpatrick, who wore a baseball cap embroider...",0,NaN,136,"['Kirkpatrick', 'wear', 'baseball', 'cap', 'em...",Kirkpatrick who wore a baseball cap embroidere...
1,2012-10-16 00:00:00,10:04:30-05:00,#<e>obama</e> debates that Cracker Ass Cracker...,1,NaN,88,"['obama', 'debate', 'Cracker', 'Ass', 'Cracker...",obama debates that Cracker Ass Cracker tonight...
2,2012-10-16 00:00:00,09:50:08-05:00,@Hollivan @hereistheanswer Youre missing the ...,0,NaN,140,"['Hollivan', 'hereistheanswer', 'Youre', 'miss...",Hollivan hereistheanswer Youre missing the poi...
3,2012-10-16 00:00:00,10:00:16-05:00,I was raised as a Democrat left the party yea...,-1,NaN,146,"['raise', 'Democrat', 'leave', 'party', 'years...",I was raised as a Democrat left the party year...
4,2012-10-16 00:00:00,09:48:07-05:00,The <e>Obama camp</e> can't afford to lower ex...,0,NaN,124,"['Obama', 'camp', 'ca', 'afford', 'lower', 'ex...",The Obama camp ca n't afford to lower expectat...


In [9]:
#Convert tweet_clst to string
df['tweet_clst'] = df['tweet_clst'].astype(str)

# The maximum number of words to be used. (most frequent)
MAX_VOCAB_SIZE = 50000
# Max number of words in each tweet.
MAX_SEQUENCE_LENGTH = 75
# This is fixed.
EMBEDDING_DIM = 150

In [21]:
from sklearn.model_selection import train_test_split
RANDOM_SEED = 42
# split 90-10 train-test
df_train, df_test = train_test_split(df, test_size=0.1, random_state=RANDOM_SEED)

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [23]:
tokenizer = Tokenizer(num_words= MAX_VOCAB_SIZE, filters='#$%&()*+<=>@[\\]^_`{|}~\t\n', lower=True)
tokenizer.fit_on_texts(df_train['tweet_clst'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 18759 unique tokens.


In [24]:
X = tokenizer.texts_to_sequences(df_train['tweet_clst'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)


Shape of data tensor: (10145, 75)


In [25]:
Y = pd.get_dummies(df_train['class']).values
print('Shape of label tensor:', Y.shape)


Shape of label tensor: (10145, 3)


In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout

model = Sequential()
model.add(Embedding(MAX_VOCAB_SIZE, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(LSTM(EMBEDDING_DIM, dropout = 0.2))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

# run for small number of epochs then save 
epochs = 10

history = model.fit(X, Y, epochs=epochs)


Epoch 1/10
318/318 [==============================] - 16s 50ms/step - loss: 0.5682 - accuracy: 0.5337
Epoch 2/10
318/318 [==============================] - 16s 49ms/step - loss: 0.4174 - accuracy: 0.7102
Epoch 3/10
318/318 [==============================] - 16s 50ms/step - loss: 0.2513 - accuracy: 0.8440
Epoch 4/10
318/318 [==============================] - 16s 50ms/step - loss: 0.1516 - accuracy: 0.9103
Epoch 5/10
318/318 [==============================] - 16s 50ms/step - loss: 0.1002 - accuracy: 0.9439
Epoch 6/10
318/318 [==============================] - 16s 49ms/step - loss: 0.0729 - accuracy: 0.9603
Epoch 7/10
318/318 [==============================] - 17s 53ms/step - loss: 0.0533 - accuracy: 0.9706
Epoch 8/10
318/318 [==============================] - 16s 52ms/step - loss: 0.0435 - accuracy: 0.9744
Epoch 9/10
318/318 [==============================] - 17s 53ms/step - loss: 0.0362 - accuracy: 0.9804
Epoch 10/10
318/318 [==============================] - 17s 53ms/step - loss: 0.028

In [30]:
X_test = tokenizer.texts_to_sequences(df_test['tweet_clst'].values)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)

Y_Test = pd.get_dummies(df_test['class']).values
print('Shape of label tensor:', Y_Test.shape)

loss, accuracy = model.evaluate(X_test, Y_Test)

print("loss",loss)
print("accuracy",accuracy)



Shape of data tensor: (1128, 75)
Shape of label tensor: (1128, 3)
36/36 [==============================] - 0s 4ms/step - loss: 1.5411 - accuracy: 0.5390
loss 1.541121482849121
accuracy 0.5390070676803589


In [37]:
from sklearn.metrics import confusion_matrix, classification_report
predictions = model.predict(X_test)

# get the class with highest probability for each sample
y_pred = np.argmax(predictions, axis=-1) - 1

print(classification_report(df_test['class'], y_pred))

              precision    recall  f1-score   support

          -1       0.59      0.58      0.59       455
           0       0.48      0.54      0.51       380
           1       0.53      0.47      0.50       293

    accuracy                           0.54      1128
   macro avg       0.54      0.53      0.53      1128
weighted avg       0.54      0.54      0.54      1128

